In [2]:
# Bloque 1: Carga de librerías y entorno
from dotenv import load_dotenv
from logger import setup_logger
from Config import RUTA_ORIGEN, RUTA_DESTINO, SEPARADOR, TABLA_DESTINO
from imports import os, glob, polars as pl, pandas as pd, tqdm, openpyxl, requests

load_dotenv()
logger = setup_logger()
logger.info("Inicio del pipeline")


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
# Bloque 2: Exploración de archivos
xlsx_files = glob.glob(f"{RUTA_ORIGEN}/*.xlsx")
print(f"Archivos encontrados: {len(xlsx_files)}")
for f in xlsx_files[:5]:  # muestra los primeros 5
    print(f.split("/")[-1])

In [ ]:
# Bloque 3: Conversión XLSX → CSV
from functions import convertir_archivos_xlsx_a_csv

convertir_archivos_xlsx_a_csv(ruta_origen=RUTA_ORIGEN, ruta_destino=RUTA_DESTINO)
logger.info("Conversión de archivos completada")

In [ ]:
# Bloque 4: Limpieza y normalización
from functions import limpiar_datos_csv

csv_files = glob.glob(f"{RUTA_DESTINO}/*.csv")
dfs_limpios = [limpiar_datos_csv(path, separador=SEPARADOR) for path in csv_files]
logger.info(f"Archivos limpiados: {len(dfs_limpios)}")

In [ ]:
# Bloque 5: Validación de esquema contra archivo_esquemas.json
from functions import validar_contra_esquema_json

df_validados = []
for df in dfs_limpios:
    df_ok = validar_contra_esquema_json(df, "archivo_esquemas.json")
    if df_ok is not None:
        df_validados.append(df_ok)

logger.info(f"Archivos que cumplen con esquema: {len(df_validados)}")

In [ ]:
# Bloque 6: Consolidación
df_consolidado = pl.concat(df_validados, how="diagonal")
print(df_consolidado.shape)
logger.info(f"DataFrame consolidado con {df_consolidado.shape[0]} registros")

In [ ]:
# Bloque 7: Registro en log
columnas_vacias = [col for col in df_consolidado.columns if df_consolidado[col].null_count() > 0]
logger.info(f"Columnas con nulos: {columnas_vacias}")
logger.info("Fin del pipeline etapa 1")

In [ ]:
# Bloque 8: Visualización (opcional)
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(df_consolidado.select(pl.count("id")), bins=20)
plt.title("Distribución de registros por archivo")
plt.show()